In [155]:
%load_ext autoreload
%autoreload 2

from math import pi, log
import torch 
from torch.nn import LSTM, Linear, Module
from src.data.words import TwitterDataWords
from src.models.iaf_words import *
from src.data.common import get_loader


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [156]:
data = torch.load('../data/processed/200316_embedding.pkl')

In [157]:
twitterwords = TwitterDataWords(data)

In [158]:
data_loader = get_loader(twitterwords, 10)

In [159]:
x = next(iter(data_loader))

In [160]:
x

PackedSequence(data=tensor([[-0.0014, -0.0141, -0.0194,  ...,  0.1428,  0.0714, -0.0579],
        [-0.0314,  0.0149, -0.0205,  ...,  0.0980,  0.0893,  0.0148],
        [ 0.0468, -0.0502,  0.0332,  ...,  0.1337,  0.0135, -0.0098],
        ...,
        [-0.0206,  0.0155, -0.0024,  ...,  0.0089, -0.0294, -0.0505],
        [ 0.0460,  0.0312, -0.1643,  ...,  0.2473, -0.0198, -0.0882],
        [ 0.2235,  0.0042,  0.0651,  ...,  0.1076,  0.1302, -0.0335]]), batch_sizes=tensor([10, 10, 10, 10, 10, 10, 10, 10, 10,  9,  9,  8,  7,  6,  5,  5,  5,  5,
         5,  5,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  2,  1,  1]), sorted_indices=None, unsorted_indices=None)

In [161]:
enc = Encoder()
ar_nn = AutoRegressiveNN(
    input_dim=64+h_dim, 
    output_dim = 2*64,
    layer1_dim = 200,
    layer2_dim = 200
)

In [162]:
mu, log_sigma, h = enc(x)
eps = torch.empty_like(mu).normal_()
z = log_sigma.exp() * eps + mu 
l = -torch.sum(log_sigma + 1/2 * torch.pow(eps,2) + 1/2 * log(2*pi))
T = 3
for t in range(T):
    m, s = ar_nn(z, h)
    sigma = s.sigmoid()
    z = sigma * z + (1 - sigma) * m
    l = l - sigma.log().sum()

In [163]:
class IAF(Module):
    def __init__(self, T, input_dim, output_dim, layer1_dim, layer2_dim):
        
        super(IAF, self).__init__()
        self.T = T,
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.layer1_dim = layer1_dim
        self.layer2_dim = layer2_dim
        
        self.ar_nn = AutoRegressiveNN(
            input_dim=self.input_dim, 
            output_dim=self.output_dim,
            layer1_dim=self.layer1_dim,
            layer2_dim=self.layer2_dim
        )
        
        
    def forward(self, mu, log_sigma, h):
        eps = torch.empty_like(mu).normal_()
        z = log_sigma.exp() * eps + mu 
        l = -torch.sum(log_sigma + 1/2 * torch.pow(eps,2) + 1/2 * log(2*pi))
        T = 3
        for t in range(T):
            m, s = ar_nn(z, h)
            sigma = s.sigmoid()
            z = sigma * z + (1 - sigma) * m
            l = l - sigma.log().sum()
        
        return z, l

In [164]:
iaf = IAF(
    T = 3,
    input_dim=64+h_dim, 
    output_dim = 2*64,
    layer1_dim = 200,
    layer2_dim = 200
)

In [167]:
z, l = iaf(*enc(x))
